In [ ]:
# 기본 라이브러리
import datetime
import time
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

# 그림 사이즈 설정
plt.rc('figure', figsize=(12,8))
font_fname = 'c:/windows/fonts/gulim.ttc' # 윈도우용
#font_fname = '/System/Library/Fonts/Supplemental/AppleGothic.ttf' # 맥용
font_name = font_manager.FontProperties(fname=font_fname).get_name()
rc('font', family=font_name)

## API를 통한 데이터 수집 - GetOldTweets3 사용 (변경)

In [ ]:
# GetOldTweet3 사용 준비
try:
    import GetOldTweets3 as got
except:
    !pip install GetOldTweets3
    import GetOldTweets3 as got

In [ ]:
# 특정 검색어가 포함된 트윗 검색하기 (quary search)

# 검색어 설정
#keyword='신한생명 OR 신한금융'
#filename="shinhan"
keyword='삼성생명'
filename="samsung"

# 수집 기간 설정
start_date = "2020-06-01"
end_date = "2020-06-13"

# 트윗 수집 기준 정의
tweetCriteria = got.manager.TweetCriteria().setQuerySearch(keyword)\
                    .setSince(start_date)\
                    .setUntil(str((pd.to_datetime(end_date) + pd.DateOffset(days=1)).date()))\
                    .setMaxTweets(-1)

print(tweetCriteria)
print("Collecting data start..") # 수집 시작 메세지 출력
start_time = time.time()         # 수집 시작 시간 측정

# 트윗 데이터 수집 명령!!!
tweets = got.manager.TweetManager.getTweets(tweetCriteria) 

print(tweets)
# print("Collecting data end..")   # 수집 종료 메세지 출력
# end_time = time.time()           # 수집 종료 시간 측정

# # 수집 결과 출력
# print("Duration: {0:0.2f} Seconds".format( (end_time - start_time)) ) # 시간
# print("Number of tweets: " + str(len(tweets))) # 개수

In [ ]:
# 읽어온 파일로 Dataframe 만들기
# 트윗 리스트 초기화
tweet_list = []

# for 문을 통해서 각 트윗을 구성
for tweet in tweets:  
    # 개별 트윗에서 데이터 가져오기 
    username = tweet.username       # 사용자 이름
    link = tweet.permalink          # 링크
    content = tweet.text            # 내용
    tweet_timestamp = tweet.date    # 날짜+시간
    tweet_date = tweet_timestamp.strftime("%Y-%m-%d") # 날짜
    tweet_time = tweet_timestamp.strftime("%H:%M:%S") # 시간
    retweets = tweet.retweets       # 리트윗
    favorites = tweet.favorites     # 추천
    hashtags = tweet.hashtags       # 해시태그
     
    # 개별 데이터 결합
    info_list = [tweet_timestamp, tweet_date, tweet_time, username, content, link, retweets, favorites, hashtags]

    # 리스트에 데이터 추가
    tweet_list.append(info_list)

    
# tweet_list에서 DataFrame 생성
twitter_df = pd.DataFrame(tweet_list, 
                          columns = ["timestamp", "date", "time", "user_name", "text", "link", 
                                     "retweet_counts", "favorites_counts","hashtags"]
                         )
twitter_df.head()

In [ ]:
# 파일 저장
twitter_df.to_csv("tweet/"+filename+"_{}_to_{}.csv".format(start_date, end_date), index=False)

## 파일 읽고 분석하기

# [Lab] tweet에 대한 분석

In [ ]:
# 파일 이름 설정
filename="shinhan"
#filename="samsung"
start_date = "2020-01-01"
end_date = "2020-06-13"

df_tweet = pd.read_csv("tweet/"+filename+'_{}_to_{}.csv'.format(start_date, end_date))
df_tweet['timestamp'] = pd.to_datetime(df_tweet['timestamp'])
df_tweet = df_tweet.set_index('timestamp')

df_tweet.info(10)

In [ ]:
# 그림 사이즈 설정
plt.rc('figure', figsize=(12,8))

In [ ]:
# 날짜별 트윗 개수 그리기 (파레토 차트)
 

# 그래프 꾸미기
plt.title("Tweets mentioning keywords in time series")
plt.ylabel("# of tweets")

In [ ]:
# 날짜별 트윗 개수 그리기 (날짜 정렬)
 

# resampling 사용 가능
# df_tweet.resample('D').size().plot(kind='bar')

In [ ]:
# 주별 트윗 개수 그리기 (날짜 정렬)
df_tweet.resample('W-MON').size().plot(kind='bar')

In [ ]:
# 월별 트윗 개수 그리기 (날짜 정렬)
df_tweet.resample('MS').size().plot(kind='bar')

## 유저 분석

In [ ]:
df_tweet["user_name"].value_counts()

In [ ]:
df_tweet["user_name"].value_counts().plot(kind='barh')

In [ ]:
                                                                   # 트윗 개수 5개 이상인 사람
 

In [ ]:
df1["user_name"].value_counts().plot(kind='barh')

## 유저 정보 분석 -- 에러

In [ ]:
#username="hangugbj"
username="Park_Youngsun"
#username="jhhan317"

#username="sinojo21"
#username="ipopou"

# 유저의 최근 트윗 개수 가져오기 (최대 20개) -- 에러 (GetOldTweets3)
tweetCriteria = got.manager.TweetCriteria().setUsername(username)\
                                           .setTopTweets(True)\
                                           .setMaxTweets(20)

tweets = got.manager.TweetManager.getTweets(tweetCriteria)

In [ ]:
# 트윗 내용 보기
for tweet in tweets:
    print(tweet.date, tweet.text)

## 해시태그 분석

In [ ]:
# 해시태그 별로 트윗 개수
df_tweet['hashtags'].value_counts().plot(kind='barh')

In [ ]:
# Unique한 해시 태크 출력
df_tweet['hashtags'].unique()

In [ ]:
# 해시 태크에서 '' 제거
 

In [ ]:
# 해시 태그별 트윗 분포
df_tweet[df_tweet["hashtags"]!='']['hashtags'].value_counts().plot(kind='barh')

In [ ]:
# 해시태그로 검색
 